In [2]:
# Import random shit
import numpy as np
import matplotlib.pyplot as plt

# Get that yum yum sklearn
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import TfidfTransformer as TFT

# Cross validation
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import ShuffleSplit

%matplotlib inline

In [3]:
# Load our data and shit
train_data = np.loadtxt('./training_data.txt', skiprows=1, delimiter=' ')
test_data = np.loadtxt('./test_data.txt', skiprows=1, delimiter=' ')

In [4]:
# Split training and test set into x and y
scaler = StandardScaler()
scaler_t = TFT()

x_train = scaler.fit_transform(train_data[:, 1:])
y_train = train_data[:, 0]

x_train_t = scaler_t.fit_transform(train_data[:, 1:])

x_test = scaler_t.transform(test_data)

In [5]:
params = {'penalty':['l2'],
          'C':[1.48, 1.5, 1.52, 1.54, 1.56, 1.58]}

log_reg = LogisticRegression()
cv = ShuffleSplit(n_splits=5, test_size=0.2, random_state=0)

clf = GridSearchCV(log_reg, params, verbose=1, cv=cv, n_jobs=4)
clf.fit(x_train_t, y_train)

Fitting 5 folds for each of 6 candidates, totalling 30 fits


[Parallel(n_jobs=4)]: Done  30 out of  30 | elapsed:   11.6s finished


GridSearchCV(cv=ShuffleSplit(n_splits=5, random_state=0, test_size=0.2, train_size=None),
       error_score='raise',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
       fit_params=None, iid=True, n_jobs=4,
       param_grid={'penalty': ['l2'], 'C': [1.48, 1.5, 1.52, 1.54, 1.56, 1.58]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=1)

In [6]:
clf.cv_results_

{'mean_fit_time': array([ 0.71630435,  0.93466167,  0.99430351,  0.98589845,  1.02612581,
         1.17162762]),
 'mean_score_time': array([ 0.00620475,  0.00760612,  0.0038034 ,  0.00700493,  0.00480371,
         0.00460458]),
 'mean_test_score': array([ 0.85365,  0.8534 ,  0.85335,  0.8535 ,  0.8535 ,  0.85345]),
 'mean_train_score': array([ 0.8760125,  0.876025 ,  0.8761   ,  0.8761875,  0.8762125,
         0.8761875]),
 'param_C': masked_array(data = [1.48 1.5 1.52 1.54 1.56 1.58],
              mask = [False False False False False False],
        fill_value = ?),
 'param_penalty': masked_array(data = ['l2' 'l2' 'l2' 'l2' 'l2' 'l2'],
              mask = [False False False False False False],
        fill_value = ?),
 'params': [{'C': 1.48, 'penalty': 'l2'},
  {'C': 1.5, 'penalty': 'l2'},
  {'C': 1.52, 'penalty': 'l2'},
  {'C': 1.54, 'penalty': 'l2'},
  {'C': 1.56, 'penalty': 'l2'},
  {'C': 1.58, 'penalty': 'l2'}],
 'rank_test_score': array([1, 5, 6, 2, 2, 4]),
 'split0_test_score

In [7]:
log_reg_y = clf.predict(x_test)
log_reg_probs = clf.predict_proba(x_test)

In [8]:
def convert(y_test, filename):
    """
    Converts the y_test vector to submission file and saves to filename
    
    Inputs:
        y_test: A (N, ) array with the predicted results for the test set
        filename: String name of file
        
    Outputs:
        None
    """
    
    text_file = np.array([[i+1, int(y_test[i])] for i in range(len(y_test))])
    np.savetxt(filename, text_file, fmt='%d', delimiter=',')
    
    f = open(filename,'r+')
    lines = f.readlines() # read old content
    f.seek(0) # go back to the beginning of the file
    f.write('Id,Prediction\n') # write new content at the beginning
    for line in lines: # write old content after new
        f.write(line)
    f.close()

In [9]:
convert(log_reg_y, 'log_reg.txt')
np.savetxt('log_reg_probs.txt', log_reg_probs)